In [1]:
import json
from pathlib import Path

from thermalnetwork.ground_heat_exchanger import GHE

In [2]:
geojson_path = Path.cwd().parent / "demos" / "sdk_output_skeleton_1_ghe" / "network.geojson"
with open(geojson_path) as f:
    geojson_data = json.load(f)

In [3]:
geojson_path = Path.cwd().parent / "demos" / "sdk_output_skeleton_2_ghe_sequential" / "network.geojson"
with open(geojson_path) as f:
    geojson_data2 = json.load(f)

In [4]:
geojson_path = Path.cwd().parent / "demos" / "sdk_output_skeleton_2_ghe_staggered" / "network.geojson"
with open(geojson_path) as f:
    geojson_data2_staggered = json.load(f)

In [ ]:
# code for line 404 of network.py to generate ghe_data.json

# ground_loads = np.array(network_load_per_area) * ghe_area
# # tolist() because writing json requires a python list, not a numpy array
# self.network[i].json_data["loads"]["ground_loads"] = ground_loads.tolist()

# sum_of_ground_loads = sum(self.network[i].json_data["loads"]["ground_loads"])
# print("json data")
# print(self.network[i].json_data)

# print("ghe_input:")
# print(sum_of_ground_loads)
# with open("ghe_data.json", "w", encoding="utf-8") as f:
#     json.dump(self.network[i].json_data, f, ensure_ascii=False, indent=4)

In [ ]:
# CLI command to use with the above code to generate ghe_data.json:
# thermalnetwork -y demos/sdk_output_skeleton_2_ghe_sequential/run/baseline_scenario/ghe_dir/sys_params.json -s demos/sdk_output_skeleton_2_ghe_sequential/run/baseline_scenario -f demos/sdk_output_skeleton_2_ghe_sequential/network.geojson -o cli_test  # noqa: E501

In [8]:
ghe_data_path = Path.cwd().parent / "ghe_data.json"
ghe_data: dict = json.loads(ghe_data_path.read_text())

In [9]:
ghe = GHE(ghe_data)

In [10]:
total_ground_loads = sum(ghe_data["properties"]["loads"]["ground_loads"])

In [11]:
ghe.ghe_size(total_ground_loads, Path.cwd())

[10:56:22] INFO     GHE_SIZE with total_space_loads: -208067140.49999994                ]8;id=283121;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=869592;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#29\29]8;;\

           DEBUG    creating directory:                                                 ]8;id=371;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=279262;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#78\78]8;;\
                    /Users/nmoore/github/ThermalNetwork/topology/test_2                                            

           INFO     saving loads to:                                                    ]8;id=382509;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=418021;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#83\83]8;;\
                    /Users/nmoore/github/ThermalNetwork/topology/test_2/ground_loads.cs                            
                    v                                                                                              

           DEBUG    loads saved to csv file                                             ]8;id=964929;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=926994;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#85\85]8;;\

[10:58:20] DEBUG    design found                                                        ]8;id=302058;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=207996;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#88\88]8;;\

           DEBUG    results prepared for writing to output directory                    ]8;id=204751;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=118020;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#90\90]8;;\

           DEBUG    output written to output directory                                  ]8;id=845019;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=53817;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#93\93]8;;\

           DEBUG    Done writing output                                                 ]8;id=120360;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py\ground_heat_exchanger.py]8;;\:]8;id=730874;file:///Users/nmoore/github/ThermalNetwork/thermalnetwork/ground_heat_exchanger.py#96\96]8;;\

134.24113385043586

In [5]:
# THIS IS GOOD
def find_startloop_feature_id(features):
    for feature in features:
        if feature["properties"].get("is_ghe_start_loop") == "true":
            start_feature_id = feature["properties"].get("buildingId") or feature["properties"].get("DSId")
            return start_feature_id
    return None


def get_connected_features(geojson_data):
    features = geojson_data["features"]
    connectors = [feature for feature in features if feature["properties"]["type"] == "ThermalConnector"]
    connected_features = []

    # get the id of the building or ds from the thermaljunction that has startloop: true
    startloop_feature_id = find_startloop_feature_id(features)

    # Start with the first connector
    start_feature_id = connectors[0]["properties"]["startFeatureId"]
    connected_features.append(start_feature_id)

    while True:
        next_feature_id = None
        for connector in connectors:
            if connector["properties"]["startFeatureId"] == connected_features[-1]:
                next_feature_id = connector["properties"]["endFeatureId"]
                break

        if next_feature_id:
            connected_features.append(next_feature_id)
            if next_feature_id == start_feature_id:
                break
        else:
            break

    # Filter and return the building and district system features
    connected_objects = []
    for feature in features:
        feature_id = feature["properties"]["id"]
        if feature_id in connected_features and feature["properties"]["type"] in ["Building", "District System"]:
            connected_objects.append(
                {
                    "id": feature_id,
                    "type": feature["properties"]["type"],
                    "name": feature["properties"].get("name", ""),
                    "start_loop": "true" if feature_id == startloop_feature_id else None,
                }
            )

    return connected_objects


def reorder_connected_features(features):
    while features[0].get("start_loop") != "true":
        features.append(features.pop(0))
    return features

In [6]:
connected_features = get_connected_features(geojson_data2)

In [7]:
connected_features = get_connected_features(geojson_data2)
for feature in connected_features:
    print(feature)

{'id': '8', 'type': 'Building', 'name': 'Hospital 1', 'start_loop': None}
{'id': '9', 'type': 'Building', 'name': 'Hospital 2', 'start_loop': None}
{'id': '10', 'type': 'Building', 'name': 'Mixed use 2', 'start_loop': None}
{'id': 'dd69549c-ecfc-4245-96dc-5b6127f34f46', 'type': 'District System', 'name': 'New District System_1', 'start_loop': 'true'}
{'id': '47fd01d3-3d72-46c0-85f2-a12854783764', 'type': 'District System', 'name': 'New District System_2', 'start_loop': None}


In [8]:
reordered_features = reorder_connected_features(connected_features)

In [9]:
reordered_features

[{'id': 'dd69549c-ecfc-4245-96dc-5b6127f34f46',
  'type': 'District System',
  'name': 'New District System_1',
  'start_loop': 'true'},
 {'id': '47fd01d3-3d72-46c0-85f2-a12854783764',
  'type': 'District System',
  'name': 'New District System_2',
  'start_loop': None},
 {'id': '8', 'type': 'Building', 'name': 'Hospital 1', 'start_loop': None},
 {'id': '9', 'type': 'Building', 'name': 'Hospital 2', 'start_loop': None},
 {'id': '10', 'type': 'Building', 'name': 'Mixed use 2', 'start_loop': None}]